In [ ]:
# Importing necessary modules from IPython to work with Jupyter notebook features
from IPython import get_ipython
from IPython.display import display

# Importing type hinting support for optional types and lists
from typing import Optional, List

# Importing required libraries
import cv2 # OpenCV for computer vision operations
import supervision as sv # Supervision library for annotation and visualization
import numpy as np # NumPy for numerical operations

In [ ]:
from sports.configs.football import FootballPitchConfiguration # Importing football pitch configuration settings for visualization or analysis

In [ ]:
def draw_pitch(
    config: FootballPitchConfiguration,
    background_color: sv.Color = sv.Color(34, 139, 34), # Default green background
    line_color: sv.Color = sv.Color.WHITE, # Default white lines
    padding: int = 50, # Padding around the pitch in pixels
    line_thickness: int = 4, # Thickness of lines
    point_radius: int = 8, # Radius for penalty spot points
    scale: float = 0.1 # Scaling factor for pitch dimensions
) -> np.ndarray:
    """
    Draws a football pitch with specified dimensions, colors, and scale.

    Args:
        config (FootballPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        background_color (sv.Color, optional): Color of the pitch background.
            Defaults to sv.Color(34, 139, 34).
        line_color (sv.Color, optional): Color of the pitch lines.
            Defaults to sv.Color.WHITE.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        line_thickness (int, optional): Thickness of the pitch lines in pixels.
            Defaults to 4.
        point_radius (int, optional): Radius of the penalty spot points in pixels.
            Defaults to 8.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.

    Returns:
        np.ndarray: Image of the football pitch.
    """

    # Scale pitch dimensions and relevant elements
    scaled_width                  = int(config.width * scale)
    scaled_length                 = int(config.length * scale)
    scaled_circle_radius          = int(config.centre_circle_radius * scale)
    scaled_penalty_spot_distance = int(config.penalty_spot_distance * scale)

    # Create background image filled with the pitch color
    pitch_image = np.ones(
        (scaled_width + 2 * padding,
         scaled_length + 2 * padding, 3),
        dtype=np.uint8
    ) * np.array(background_color.as_bgr(), dtype=np.uint8)

    # Draw pitch lines based on vertex connections
    for start, end in config.edges:
        point1 = (
            int(config.vertices[start - 1][0] * scale) + padding,
            int(config.vertices[start - 1][1] * scale) + padding
        )
        point2 = (
            int(config.vertices[end - 1][0] * scale) + padding,
            int(config.vertices[end - 1][1] * scale) + padding
        )
        cv2.line(
            img=pitch_image,
            pt1=point1,
            pt2=point2,
            color=line_color.as_bgr(),
            thickness=line_thickness
        )

    # Draw the center circle
    centre_circle_center = (
        scaled_length // 2 + padding,
        scaled_width  // 2 + padding
    )
    cv2.circle(
        img=pitch_image,
        center=centre_circle_center,
        radius=scaled_circle_radius,
        color=line_color.as_bgr(),
        thickness=line_thickness
    )

    # Draw the penalty spots
    penalty_spots = [
        (
            scaled_penalty_spot_distance + padding,
            scaled_width // 2 + padding
        ),
        (
            scaled_length - scaled_penalty_spot_distance + padding,
            scaled_width // 2 + padding
        )
    ]
    for spot in penalty_spots:
        cv2.circle(
            img=pitch_image,
            center=spot,
            radius=point_radius,
            color=line_color.as_bgr(),
            thickness=-1  # Filled circle
        )

    # Return the final image of the pitch
    return pitch_image

In [ ]:
def draw_points_on_pitch(
    config: FootballPitchConfiguration,
    xy: np.ndarray,
    face_color: sv.Color = sv.Color.RED, # Color of the point fill
    edge_color: sv.Color = sv.Color.BLACK, # Color of the point edge/border
    radius: int = 10, # Radius of each point in pixels
    thickness: int = 2, # Thickness of the edge border
    padding: int = 50, # Padding around the pitch in pixels
    scale: float = 0.1, # Scaling factor for pitch dimensions
    pitch: Optional[np.ndarray] = None # Optional pre-drawn pitch to draw points on
) -> np.ndarray:
    """
    Draws points on a football pitch.

    Args:
        config (FootballPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        xy (np.ndarray): Array of points to be drawn, with each point represented by
            its (x, y) coordinates.
        face_color (sv.Color, optional): Color of the point faces.
            Defaults to sv.Color.RED.
        edge_color (sv.Color, optional): Color of the point edges.
            Defaults to sv.Color.BLACK.
        radius (int, optional): Radius of the points in pixels.
            Defaults to 10.
        thickness (int, optional): Thickness of the point edges in pixels.
            Defaults to 2.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw points on.
            If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the football pitch with points drawn on it.
    """

    # If no pitch image is provided, create one using draw_pitch
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    # Draw each point on the pitch
    for point in xy:
        scaled_point = (
            int(point[0] * scale) + padding,
            int(point[1] * scale) + padding
        )
        # Draw filled circle (face color)
        cv2.circle(
            img=pitch,
            center=scaled_point,
            radius=radius,
            color=face_color.as_bgr(),
            thickness=-1
        )
        # Draw edge/border circle (edge color)
        cv2.circle(
            img=pitch,
            center=scaled_point,
            radius=radius,
            color=edge_color.as_bgr(),
            thickness=thickness
        )

    # Return the pitch image with drawn points
    return pitch

In [ ]:
def draw_paths_on_pitch(
    config: FootballPitchConfiguration,
    paths: List[np.ndarray], # List of paths (each path is an array of (x, y) points)
    color: sv.Color = sv.Color.WHITE, # Color of the paths (default: white)
    thickness: int = 2, # Line thickness in pixels
    padding: int = 50, # Padding around the pitch
    scale: float = 0.1, # Scaling factor for pitch size
    pitch: Optional[np.ndarray] = None # Optional existing pitch to draw on
) -> np.ndarray:
    """
    Draws paths on a football pitch.

    Args:
        config (FootballPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        paths (List[np.ndarray]): List of paths, where each path is an array of (x, y)
            coordinates.
        color (sv.Color, optional): Color of the paths.
            Defaults to sv.Color.WHITE.
        thickness (int, optional): Thickness of the paths in pixels.
            Defaults to 2.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw paths on.
            If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the football pitch with paths drawn on it.
    """

    # Create a new pitch if not provided
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    # Iterate over all paths
    for path in paths:
        # Scale and shift each point according to padding and scale
        scaled_path = [
            (
                int(point[0] * scale) + padding,
                int(point[1] * scale) + padding
            )
            for point in path if point.size > 0
        ]

        # Skip if path has fewer than 2 points
        if len(scaled_path) < 2:
            continue

        # Draw lines between consecutive points in the path
        for i in range(len(scaled_path) - 1):
            cv2.line(
                img=pitch,
                pt1=scaled_path[i],
                pt2=scaled_path[i + 1],
                color=color.as_bgr(),
                thickness=thickness
            )

    # Return final image with drawn paths
    return pitch

In [ ]:
def draw_pitch_voronoi_diagram(
    config: FootballPitchConfiguration,
    team_1_xy: np.ndarray, # (x, y) positions for team 1
    team_2_xy: np.ndarray, # (x, y) positions for team 2
    team_1_color: sv.Color = sv.Color.RED, # Control area color for team 1
    team_2_color: sv.Color = sv.Color.WHITE, # Control area color for team 2
    opacity: float = 0.5, # Opacity for Voronoi overlay
    padding: int = 50, # Padding around pitch
    scale: float = 0.1, # Scaling factor for pitch size
    pitch: Optional[np.ndarray] = None # Optional background pitch image
) -> np.ndarray:
    """
    Draws a Voronoi diagram on a football pitch representing the control areas of two
    teams.

    Args:
        config (FootballPitchConfiguration): Configuration object containing the
            dimensions and layout of the pitch.
        team_1_xy (np.ndarray): Array of (x, y) coordinates representing the positions
            of players in team 1.
        team_2_xy (np.ndarray): Array of (x, y) coordinates representing the positions
            of players in team 2.
        team_1_color (sv.Color, optional): Color representing the control area of
            team 1. Defaults to sv.Color.RED.
        team_2_color (sv.Color, optional): Color representing the control area of
            team 2. Defaults to sv.Color.WHITE.
        opacity (float, optional): Opacity of the Voronoi diagram overlay.
            Defaults to 0.5.
        padding (int, optional): Padding around the pitch in pixels.
            Defaults to 50.
        scale (float, optional): Scaling factor for the pitch dimensions.
            Defaults to 0.1.
        pitch (Optional[np.ndarray], optional): Existing pitch image to draw the
            Voronoi diagram on. If None, a new pitch will be created. Defaults to None.

    Returns:
        np.ndarray: Image of the football pitch with the Voronoi diagram overlay.
    """

    # Create new pitch image if not provided
    if pitch is None:
        pitch = draw_pitch(
            config=config,
            padding=padding,
            scale=scale
        )

    # Scale pitch size
    scaled_width  = int(config.width * scale)
    scaled_length = int(config.length * scale)

    # Create an empty overlay image for Voronoi regions
    voronoi = np.zeros_like(pitch, dtype=np.uint8)

    # Convert team colors to BGR NumPy arrays
    team_1_color_bgr = np.array(team_1_color.as_bgr(), dtype=np.uint8)
    team_2_color_bgr = np.array(team_2_color.as_bgr(), dtype=np.uint8)

    # Generate a meshgrid of all pixel positions
    y_coordinates, x_coordinates = np.indices((
        scaled_width + 2 * padding,
        scaled_length + 2 * padding
    ))

    # Adjust coordinates to remove padding offset
    y_coordinates -= padding
    x_coordinates -= padding

    # Function to compute Euclidean distances from all points to each player
    def calculate_distances(xy, x_coordinates, y_coordinates):
        return np.sqrt(
            (xy[:, 0][:, None, None] * scale - x_coordinates) ** 2 +
            (xy[:, 1][:, None, None] * scale - y_coordinates) ** 2
        )

    # Calculate distance maps for both teams
    distances_team_1 = calculate_distances(team_1_xy, x_coordinates, y_coordinates)
    distances_team_2 = calculate_distances(team_2_xy, x_coordinates, y_coordinates)

    # Find the minimum distance to each team for every pixel
    min_distances_team_1 = np.min(distances_team_1, axis=0)
    min_distances_team_2 = np.min(distances_team_2, axis=0)

    # Create mask: True where team 1 is closer, False where team 2 is
    control_mask = min_distances_team_1 < min_distances_team_2

    # Assign colors to Voronoi regions based on control
    voronoi[control_mask]  = team_1_color_bgr
    voronoi[~control_mask] = team_2_color_bgr

    # Blend Voronoi overlay with pitch using specified opacity
    overlay = cv2.addWeighted(voronoi, opacity, pitch, 1 - opacity, 0)

    # Return final pitch with Voronoi overlay
    return overlay